<a href="https://colab.research.google.com/github/aokit/colaboratory/blob/master/20191106_1734.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
# os.listdir()
import glob
# 
r''' \Uabcdefgh '''
r'''
『C:\Program Files\7-Zip;W:\opt\Continuum\anaconda3;D:\Users\00002340\AppData\Local\GitHubDesktop\bin』
'''
r'''############################################################################
・まめちしき

『C:\Program Files\7-Zip;
  W:\opt\Continuum\anaconda3;
  D:\Users\00002340\AppData\Local\GitHubDesktop\bin』

コメントとして『３シングルクォート』を使うのは、ソースコードのその場所に
改行込みの文字列をおいているだけのこと。
実際に値として代入することもできるし。
なので、その中に『バックスラッシュＵ』が入るとユニコードとして評価しようとし
それができないとエラーになってしまう。
そこで、いっそのことコメントをロウ文字扱いの文字列
(rで始める３シングルクォート)にしてしまうとよい。ちなみに、シングルクォートの
ロウ文字では、文字列末が奇数個のバックスラッシュだとエラーになるが、
３シングルコートの中の行末の場合なら大丈夫みたい。こんな感じ→\
# ただ、３シングルクォートの直前が『\』だとさすがにだめ。
'ここ'
↓
'''#############################################################################
# os.chdir('sample_data')
# glob.glob('*')

# %cd
if False:
  True
else:
  print("faaalse")
  # import os
  filepath = './dir/subdir/filename.ext'
  print(os.path.dirname(filepath))
  print(os.getcwd())

In [0]:
r'''############################################################################
検証用のデータのセットアップ（google driveのマウント手順を含む）
'''#############################################################################
# googleドライブをマウントするときにはこのセルを実行する。
# このセルは左ペインの『ドライブをマウント』で自動的に追加された。
# このセルを実行すると、URLが表示されて、それをクリックして、認証すると
# クレデンシャルが現れるのでそれを入力するとマウントが完了する。便利。
# ちなみに、AAE.zip
# を展開して、このセルの以下のをやってみる感じ。
import os
if not ( os.path.isdir('/content/drive/My Drive')) :
  from google.colab import drive
  drive.mount('/content/drive')
#
# if not ( os.path.isdir('../AAE')) :
#   os.mkdir('../AAE')
#
import shutil
import zipfile
if ( os.path.isdir('../PAAE')) :
  shutil.rmtree('../PAAE')
if ( os.path.isdir('/content/drive/My Drive/AT/実験室/PAAE')) :
  shutil.rmtree('/content/drive/My Drive/AT/実験室/PAAE')
os.mkdir('/content/drive/My Drive/AT/実験室/PAAE')
with zipfile.ZipFile('/content/drive/My Drive/AT/実験室/AAE.zip') as existing_zip:
  existing_zip.extractall('/content/drive/My Drive/AT/実験室/PAAE')
if ( os.path.isdir('/content/drive/My Drive/AT/実験室/AAE')) :
  shutil.rmtree('/content/drive/My Drive/AT/実験室/AAE')
os.mkdir('/content/drive/My Drive/AT/実験室/AAE')
# shutil.copy2('/content/drive/My Drive/AT/実験室/1BNAQ_AAE.zip','/content/drive/My Drive/AT/実験室/PAAE')

In [0]:
# -*- code:utf-8 -*-

r'''############################################################################
tfdiff4.py
----------
netNACCSで自動保存に失敗した場合に、必要となる、
自動保存ファイル名の形式の復元機能
を組み込み。
'''#############################################################################

r'''############################################################################
Winactorのシナリオで失敗して再実行する場合

  確認済電文フォルダ▼FnTFF =
  　\\atsvfs.at-ad.ntt-at.co.jp\経企部\経輸出\経輸出\
  　　００　経輸出共用（経企部ＮＡＳ）\６　NACCS\
  　　　021_b_輸出許可通知書（テキスト形式）\AAE▼

に入れた電文ファイルを

  輸出許可通知書の未確認（確認対象）電文フォルダ▼FnP_AAE=r't:\P_AAE'▼

にもどして、実行。
＜次回更新案＞
この、巻き戻し操作は、一発でできるといいな。ログを消す必要はないが、ログを使って
できるはずだから。

いや、その逆・・・

この tfdiff2.py 自体が、Winactorのシナリオが正しく最後まで走ったことを
確認してから、ファイルを移動させるようにする、という手がある。

このスクリプトの中では、txt -> pxt と拡張子を変えることまでしかやらず、
移動しない。その代わりに、移動のための bat ファイルを作成しておく。
Winactor のシナリオの最後で、その移動のための bat ファイルを実行する。
・・・と、いう方法。

を組み込んで tfdiff3.py とした。

'''#############################################################################


r'''############################################################################

◆実行のしかた：引数なしで『python tfdiff4.py』で実行する。
◆処理の概要：
輸出許可通知書の未確認（確認対象）電文フォルダ▼FnP_AAE=r't:\P_AAE'▼にある
電文ファイルから、電文リストファイル（すべて／承認区分ごと）を作成し、
電文ファイルの拡張子を .pxt に変更して格納先へ移動する。
電文ファイルは、netNACCSから取り出されるテキストファイルであり、netNACCSに
よって許可通知書を得ることが可能なファイルである。
文字コードは shift_jis である。
Windows では既定のままで読み込みが可能であるが、Linuxでの検証も並行して
実施しており、そのため、明示的して shift_jis で読み込む。

python は、以下にパス（00002340のユーザ環境変数の PATH の値）を通してある。
『C:\Program Files\7-Zip;
  W:\opt\Continuum\anaconda3;
  D:\Users\00002340\AppData\Local\GitHubDesktop\bin』
  ↑この中の『W:\opt\Continuum\anaconda3』のところ。
  ・・・・・
  なお『￥Ｕ』（の半角）は、unicode の表記（\U00000000　のように）
  をなしていないためのエラーとなる。そのためコメントの中でも
  『全角の円記号』で置換するとよい。
  もしくは、このコメントのようにｒで始める３シングルコートで始めるといい。

・まめちしき
コメントとして『３シングルクォート』を使うのは、ソースコードのその場所に
改行込みの文字列をおいているだけのことになる。
実際に値として代入することもできるし。
なので、その中に『バックスラッシュＵ』が入るとユニコードとして評価しようとし
それができないとエラーになってしまう。
そこで、いっそのことコメントをロウ文字扱いの文字列
(rで始める３シングルコード)にしてしまうとよい。\
'''#############################################################################

# tfdiff4.py ###################################################################

def tfdiff1(FnB_0,Fn_FE,Fn_N1,Fn_NO,Fn_XX,FnTFF,FnP_AAE,
            FnCleanup,
            l_Cleanup,
            S_move,
            FnLOG,
            TFFP='*.*xt'#'1BNAQ_*'
            ):
  #
  import os
  PD = os.sep
 
  import sys
  args = sys.argv

  #### いったんここまで ####

  import shutil
  # ↑以下の繰り返しの中で電文ファイルをリネームして FnTFF に移動させるため
  #   shutil.move(old,renew)

  import glob
  # 開くべきファイルのリストを取得する
  os.chdir(FnP_AAE)
  l_B = glob.glob(TFFP)
  r'''##############↑##############################################
  拡張子にかかわらず先頭が一致するものをすべてリストアップしている。
  なので、
  ・拡張子が違うだけなら処理対象になる（.txt→.pxtのあと再処理可能）
  ・.zipファイルなどを置いてしまうとそれも処理しようとしてテキスト
  　のコードが異常であるためにエラーが発生する。
  ので注意。
  '''###############################################################

  l_B0 = []
  fl = []
  l_FE = []
  l_N1 = []
  l_NO = []
  l_XX = []
  l_SL = []
  FnSysLog = 'LOG.pxt'

  # 以下、ファイルはフルパスにしておく
  for i in range(len(l_B)):
    '''
    (最初を0行として)
    64行目（1558文字目）から『FE』『N1』『N0』が記載されている・・・はず。
    '''
    org = l_B[i].strip()
    new = os.path.splitext(org)[0] + '.pxt'
    path_t = FnP_AAE
    path_t += PD + org
    # print(path_t)
    if os.path.isfile(path_t):
      # print(path_t)
      with open(path_t,mode='r',encoding='shift_jis') as f:
        try:
          fl = f.readlines()
          # UnicodeDecodeError: が起きたかどうかで処理を分ける。
        except UnicodeDecodeError:
          print('ファイル名' + path_t + 'で UnicodeDecodeError エラー')
          # raise Exception('ファイル名' + path_t + 'でエラー')
          l_SL.append('ファイル名' + path_t + 
                      'で UnicodeDecodeError エラー' + '\n')
          # 読み込みに失敗するとバッファ変数の fl には前の結果が残ってしまって
          # いる。後続処理での整合のため無効電文として fl をクリアしておく。
          fl =[]
        else :
          new_1 = ( '1BNAQ_' + fl[0][8:15] +
                   '_' + fl[0][115:139].rstrip() + 
                   '_'+fl[0][15:27]+'ss.pxt' )
          if ( len(new) < 44 ) or ( not ( new[0:44] == new_1[0:44] )):
            new = new_1
          l_B0.append(new + '\n')
      #
      # 自動保存ファイル名の形式の復元機能のために上記の new_1 を導入。
      # Winactor による自動処理ではnetNACCSで自動保存されたファイル名を
      # 利用しているため、自動保存に失敗した場合には、Winactorでの処理が
      # うまくいかない。そこで、
      # 読み込んだファイルに基づく移動先のファイル名（new）が
      # 自動保存のフォーマットを持っていないとき（前方44文字で判断）には
      # 書き出すファイルのファイル名（移動先のファイル名）としては、
      # 自動保存のフォーマットに準じるファイル名（new_1）に変更する。
      # 例：『1BNAQ_AAE1SF3_12296123220 777336701770_201912262319ss.txt』
      #
      if len(fl) > 63 :
        # 許可承認区分（『FE』『N1』『N0』）は64行目にある
        if fl[64].startswith('FE') :
          l_FE.append(new + '\n')
        elif fl[64].startswith('N1') :
          l_N1.append(new + '\n')
        elif fl[64].startswith('NO') :
          l_NO.append(new + '\n')
        else :
          # 64行目まであれば、上記いずれかが記載されている
          # どの記載もないならば、エラーである。
          # print('Error:' + fl[64] + ':' + path_t)
          # raise Exception('電文ファイル ' + path_t + 
          #                 ' に許可承認区分の表示が見つかりません')
          # 不適切な文字コードのファイルは処理対象としなくてよい場合が
          # 通常なので、例外をあげるのではなく、ログ・ファイルに記録する。
          l_SL.append('電文ファイル ' + path_t + 
                      ' に許可承認区分の表示が見つかりません' + '\n')
      else :
        if len(fl) > 0 :
          # 輸出マニュフェスト通関申告の場合など、
          # 行数が少なく許可承認区分が記載されていない場合がある
          l_XX.append(new + '\n')
        # fl = [] でやってきたときには、電文でないとしてl_XX にも入れない。
      if len(fl) > 0 :
        # 電文ファイルの移動（有効な電文の場合のみ）
        old = path_t
        # new = os.path.splitext(path_t)[0] + '.pxt'
        renew = FnTFF + PD + new
        #
        # shutil.move(old,new)
        # shutil.move(new,renew)
        # shutil.move(old,renew)
        # l_Cleanup.append(S_move + '"' + old + '"' + ' ' + '"' + renew + '"' + '\n')
        # パスに区切りが入るとき、バックスラッシュを使っている場合に備えると
        # r'...' と記述することが必要。
        #                         ↓                       ↓
        S_tmp = S_move
        # パス区切りを強制的に『/』にする。
        S_tmp += 'r"' + old.replace(PD,'/') + '"' + ','
        S_tmp += 'r"' + renew.replace(PD,'/') + '")' + '\n'
        l_Cleanup.append(S_tmp)
        #
        # （このあと、テキストファイルに打ち出すから。
        # スクリプトの上記の部分のように、変数そのまま使うのであれば考慮しな
        # くても大丈夫だった・・・運良く）
        # また、shutil は、windows であってもパス区切りが『/』のままで
        # 機能するので、そのまま気にしないで『/』だけで構成していく手もあった
        # かもしれない・・・というか、それでないと結局だめだった。唯一の正解。
        # 
      # fl = [] で出てきたときには、電文でないとしてファイルの移動もしない。
  # 書き込むべきファイルを取り扱えるディレクトリへ移動
  os.chdir(FnTFF)
  os.mkdir(FnLOG)
  os.chdir(FnLOG)
  with open(FnB_0, mode='w') as f:
    # f.writelines(l_B0)
    # f.write('\n'.join(l_B0))  
    # f.write('\n'.join(l_B0)+'\n')  
    f.writelines(l_B0)
  with open(Fn_FE, mode='w') as f:
    f.writelines(l_FE)
  with open(Fn_N1, mode='w') as f:
    f.writelines(l_N1)
  with open(Fn_NO, mode='w') as f:
    f.writelines(l_NO)
  with open(Fn_XX, mode='w') as f:
    f.writelines(l_XX)
  with open(FnSysLog, mode='w') as f:
    f.writelines(l_SL)
  ### スクリプトの生成だけは文字コード utf-8 が必須のため明示
  with open(FnCleanup, mode='w', encoding='utf-8') as f:
    f.writelines(l_Cleanup)

# tfdiff4.py ###################################################################

def main():
  r'''##########################################################################
  輸出許可通知書電文フォルダ《１》FnTFF = '..\\AAE'
  が、InfoCageの保護対象になってしまっている。そうすると、スクリプト処理
  をかけることができない（対策調査中）
  ひとまず、
  \\atsvfs.at-ad.ntt-at.co.jp\FSUsers\00002340\W_work\transfer
  の下に
  AAE
  を作って、ここに netNACCS から電文ファイルを投入してもらう。
  FnP_AAE=r'\\atsvfs.at-ad.ntt-at.co.jp\FSUsers\00002340\W_work\transfer\P_AAE'
  FnP_AAE=r't:\P_AAE'
  FnP_AAE = 't:' + PD + 'P_AAE' + PD
  （t:\P_AAE\）
  このスクリプトが引数なしで呼び出されたときに、
  生成される確認電文リスト《２》FnB_0 = 'B_0.pxt'
  の内容にしたがって、上記のフォルダの電文ファイルを
  拡張子を '.pxt' に変えて
  FnTFF =
  　\\atsvfs.at-ad.ntt-at.co.jp\経企部\経輸出\経輸出\
  　　００　経輸出共用（経企部ＮＡＳ）\６　NACCS\
  　　　021_b_輸出許可通知書（テキスト形式）\AAE
  に書き込むことにする。
  renew = FnAAE + PD + os.path.splitext('1BNAQ_AAE1JF3_18530799220 349823164_20191031100448.txt')[0] + '.pxt'

  ⇒その場合、『あらたに追加されたファイル』を調べる必要はなかった。
  　FnP_AAE に入っている電文ファイルが、そのまま、新しいファイル。
  　また、古い電文ファイルに関しては、拡張子を無視（変換）して
  　取り扱うように改造しなくてはならない。
  　これら、未着手
  '''###########################################################################

  # tfdiff4.py #################################################################

  r'''##########################################################################
  使い方：
  ◆引数なしの場合：

  python tfdiff4.py
  ・・・・・・・・
  輸出許可通知書の未確認（確認対象）電文フォルダ▼FnP_AAE=r't:\P_AAE'▼にある
  すべての電文ファイルにもとづいて、

  確認済電文フォルダ▼FnTFF =
  　\\atsvfs.at-ad.ntt-at.co.jp\経企部\経輸出\経輸出\
  　　００　経輸出共用（経企部ＮＡＳ）\６　NACCS\
  　　　021_b_輸出許可通知書（テキスト形式）\AAE▼
  の下のログファイルのフォルダ名▼FnLOG =
  　'LOG_' + datetime.datetime.now().strftime('%Y-%m-%d_%H%M%S_%f')▼
  に、
  次のファイルを作る。
  １：すべての未確認（確認対象）電文のファイル名（拡張子書き換え後）のリスト
  ２：区分FEの・・・
  ３：区分N1の・・・
  ４：区分NOの・・・
  ５：区分XXの・・・
  ６：１の複製でファイル名に日付を与えたもの
  さらに、未確認電文フォルダのファイルを、拡張子を書き換えて
  確認済電文フォルダに移動させる。
  '''###########################################################################

  import os
  # 実行環境が Windows か Linux かを識別する差異の基準になる。
  # 文字列の明示的表記の場合は末尾が『\』だとエラーになるので
  # ↓これを使うこともできる
  PD = os.sep

  # 輸出許可通知書の未確認（確認対象）電文フォルダ
  # 確認済電文フォルダ
  # の指定；
  # osがどちらであるかによって、処理対象のフォルダのパスの表記（区切りも違う）
  # とアレンジを切り替える
  if not ( PD == '/' ) :
    #
    # 輸出許可通知書の未確認（確認対象）電文フォルダ
    PnPAE = r't:\P_AAE'
    r''' ###↑
    【要準備項目：１】InfoCageに介入されない電文フォルダを用意しておくこと
    ・・・・
    InfoCageの保護で直接読み取りが不可能にならないよう、非保護の領域を
    net NACCS から電文ファイルを書き出す先として用意しておく。
    ここでは、
    \\atsvfs1.at-ad.ntt-at.co.jp\FSUsers\00002340\W_work\transfer
    を、 windows のネットワークドライブとして『t:』にマウントしてある
    ものとしている。
    rから始まる raw文字列としているのは、Windowsのパス区切り文字が
    バックスラッシュであり、Python コードのエスケープの機能となること
    を停止させておきたいため。
    ''' ####
    # 確認済電文フォルダ
    r''' ###
    FnCleanup = PnPAE + r'\Cleanup.bat'
    l_Cleanup = []
    S_move = r'move '
    ''' ####
    FnCleanup = PnPAE + r'\Cleanup.py'
    l_Cleanup = []
    l_Cleanup.append('import shutil\n')
    S_move = r'shutil.move('
    ########
    PnTFF = r'y:\００　経輸出共用（経企部ＮＡＳ）\６　NACCS\021_b_輸出許可通知書（テキスト形式）\AAE'
    #
    # PnPAE = r'g:\user01\20191120\PAAE'
    # PnTFF = r'g:\user01\20191120\AAE'
    #
  else :
    PnPAE = r'/content/drive/My Drive/AT/実験室/PAAE'
    r''' ###
    FnCleanup = PnPAE + r'/Cleanup.sh'
    l_Cleanup = ['#!/bin/sh\n']
    S_move = r'mv '
    ''' ####
    FnCleanup = PnPAE + r'/Cleanup.py'
    l_Cleanup = []
    l_Cleanup.append('import shutil\n')
    S_move = r'shutil.move('    
    PnTFF = r'/content/drive/My Drive/AT/実験室/AAE'
    #
  #
  if not os.path.exists(PnPAE) :
    raise Exception('処理対象のパス《FnP_AAE = ' + PnPAE + '》が存在しません。')
    # quit(1)

  # 確認済電文フォルダ　に配置する
  # 処理結果（出力）ファイルの指定

  # １：すべての未確認（確認対象）電文のファイル名（拡張子書き換え後）のリスト
  FnB_0 = 'B_0.pxt'

  # ２：区分FEの電文のファイル名（拡張子書き換え後）のリスト
  # ３：区分N1の・・・
  # ４：区分NOの・・・
  # ５：区分XXの・・・
  Fn_FE = 'FE.pxt'
  Fn_N1 = 'N1.pxt'
  Fn_NO = 'NO.pxt'
  Fn_XX = 'XX.pxt'
  # ６：１の複製でファイル名に日付を与えたもの
  import datetime
  # FnLOG = 'B_' + datetime.datetime.now().strftime('%Y-%m-%d_%H%M%S_%f') + '.pxt'
  # FnTFFの中に作成するログファイルのフォルダ名（FnTFFからの相対パス）
  FnLOG = 'LOG_' + datetime.datetime.now().strftime('%Y-%m-%d_%H%M%S_%f')

  tfdiff1(FnB_0,Fn_FE,Fn_N1,Fn_NO,Fn_XX,PnTFF,PnPAE,
          FnCleanup,
          l_Cleanup,
          S_move,
          FnLOG)

  # 最後にこれをやっておく
  import shutil
  shutil.copy2(PnTFF + PD + FnLOG + PD + FnB_0, PnTFF + PD + FnB_0)
#
# 実行 #########################################################################
main()
################################################################################
r''' ###
以上を実行したあと、FnP_AAE に示されるフォルダに FnCleanup で示される
Python スクリプトのファイルが生成される。このファイルは上記の設定の
ままであれば、Cleanup.py でありその場合；
python Cleanup.py
を実行すると、電文ファイルの移動と拡張子の変更が行われる。
なお、それを colaboratory の中で確認することもできた。

電文ファイルが自動で格納できず、手動で格納した場合にも、
ファイル名が、ある程度は自動保存の形式に復元されようにした。→確認した。
自動保存の形式は、一部固定長ではない。.rstrip() で空白を削除した。
処理対象のファイルのファイル名パタンを 1BNAQ で始まるものとしていたが
自動保存の形式でない場合も対象とすることから、 *.*xt つまり、拡張子が
.txt のものも .pxt のものも、ファイル名が何で始まっても対象とすることとした。

''' ####
''''''

''

In [0]:
%cd /content/drive/My Drive/AT/実験室/PAAE
!python Cleanup.py
!ls
%cd /content/drive/My Drive/AT/実験室/AAE
!ls ../AAE

/content/drive/My Drive/AT/実験室/PAAE
Cleanup.py
/content/drive/My Drive/AT/実験室/AAE
'1BNAQ_AAE1JF3_18530799220 349823164_201910301314ss.pxt'
'1BNAQ_AAE1JF3_48319760630 8916574995_20191009172744.pxt'
'1BNAQ_AAE1JF3_48320786610 24910ATHX3P_20191015170300.pxt'
'1BNAQ_AAE1JF3_48328885240 29V24W3NXYJ_201910311652ss.pxt'
'1BNAQ_AAE1LF3_12177823810 369902701200_20191008203743.pxt'
'1BNAQ_AAE1LF3_12202755710 813522601621_20191028084415.pxt'
'1BNAQ_AAE1LF3_12208656230 1064618380_201910301914ss.pxt'
'1BNAQ_AAE1LF3_12212094960 349836450_201911011401ss.pxt'
'1BNAQ_AAE1LF3_42437351010 776615738473_20191015170226.pxt'
'1BNAQ_AAE1LF3_42437652450 NEC75290176_201910111452ss.pxt'
 B_0.pxt
 LOG_2020-01-07_002703_195759


In [0]:
%cd /content/drive/My\ Drive/AT/実験室/PAAE
!cat Cleanup.py

/content/drive/My Drive/AT/実験室/PAAE
import shutil
shutil.move(r"/content/drive/My Drive/AT/実験室/PAAE/1BNAQ_AAE1JF3_48319760630 8916574995_20191009172744.txt",r"/content/drive/My Drive/AT/実験室/AAE/1BNAQ_AAE1JF3_48319760630 8916574995_20191009172744.pxt")
shutil.move(r"/content/drive/My Drive/AT/実験室/PAAE/1BNAQ_AAE1JF3_48320786610 24910ATHX3P_20191015170300.txt",r"/content/drive/My Drive/AT/実験室/AAE/1BNAQ_AAE1JF3_48320786610 24910ATHX3P_20191015170300.pxt")
shutil.move(r"/content/drive/My Drive/AT/実験室/PAAE/1BNAQ_AAE1LF3_12177823810 369902701200_20191008203743.txt",r"/content/drive/My Drive/AT/実験室/AAE/1BNAQ_AAE1LF3_12177823810 369902701200_20191008203743.pxt")
shutil.move(r"/content/drive/My Drive/AT/実験室/PAAE/1BNAQ_AAE1LF3_12202755710 813522601621_20191028084415.txt",r"/content/drive/My Drive/AT/実験室/AAE/1BNAQ_AAE1LF3_12202755710 813522601621_20191028084415.pxt")
shutil.move(r"/content/drive/My Drive/AT/実験室/PAAE/1BNAQ_AAE1LF3_42437351010 776615738473_20191015170226.txt",r"/content/drive/My Dri

y:/００　経輸出共用（経企部ＮＡＳ）/６　NACCS/021_b_輸出許可通知書（テキスト形式）/TFDIFF/